In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/dataset.zip /content/

In [3]:
!unzip -q /content/dataset.zip -d /content/dataset

In [4]:
!ls /content/dataset

data.yaml  images  labels


In [5]:
!rm -r /content/my_yolo

rm: cannot remove '/content/my_yolo': No such file or directory


In [6]:
!mkdir -p /content/my_yolo
!cp -r /content/drive/MyDrive/my_yolo/* /content/my_yolo/

In [7]:
!ls /content/my_yolo/

dataset.py  metrics_vect.py  __pycache__  train_fixed.py
loss.py     model_res.py     test.py


In [8]:
import sys
sys.path.insert(0, "/content/my_yolo")

In [9]:
import torch
from loss import YoloLoss
from dataset import MyYoloDataset
from model_res import MyYolo_res
from train_fixed import model_training

In [ ]:
EPOCHS = 50
BATCH_SIZE = 32
LR = 1.5e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = MyYoloDataset("/content/dataset", split='train')
val_dataset = MyYoloDataset("/content/dataset", split='val')

num_classes = train_dataset.num_classes

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model = MyYolo_res(num_classes).to(DEVICE)
criterion = YoloLoss(num_classes, lambda_noobj=5.0).to(DEVICE)
# AdamW aggiunge una penalità ai pesi (weight_decay) per ridurre l'overfitting
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=5e-2)

# Scheduler: riduce il LR del 10% se la loss non migliora per 3 epoche
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [ ]:
# Avvio addestramento
train_losses, val_losses, val_f1s = model_training(
    EPOCHS, model, train_loader, val_loader, optimizer, criterion, DEVICE, scheduler
)


Epoch: 1/50 | Time: 151.2s
  Train Loss: 6.4196
  Val Loss:   4.7222 | Val Precision: 44.13% | Val Recall: 12.06% | Val F1: 18.57%
*** Modello salvato! (Loss: 4.7222) ***

Epoch: 2/50 | Time: 149.3s
  Train Loss: 4.4593
  Val Loss:   4.1344 | Val Precision: 50.14% | Val Recall: 16.85% | Val F1: 24.87%
*** Modello salvato! (Loss: 4.1344) ***

Epoch: 3/50 | Time: 151.9s
  Train Loss: 3.8533
  Val Loss:   3.6465 | Val Precision: 51.15% | Val Recall: 29.09% | Val F1: 36.82%
*** Modello salvato! (Loss: 3.6465) ***

Epoch: 4/50 | Time: 152.7s
  Train Loss: 3.4840
  Val Loss:   3.4630 | Val Precision: 57.22% | Val Recall: 33.21% | Val F1: 41.88%
*** Modello salvato! (Loss: 3.4630) ***

Epoch: 5/50 | Time: 148.9s
  Train Loss: 3.2170
  Val Loss:   3.2572 | Val Precision: 57.71% | Val Recall: 39.53% | Val F1: 46.71%
*** Modello salvato! (Loss: 3.2572) ***

Epoch: 6/50 | Time: 151.8s
  Train Loss: 3.0353
  Val Loss:   3.0292 | Val Precision: 57.66% | Val Recall: 44.24% | Val F1: 49.90%
*** Mode